## "Рекомендации на основе содержания"
* Использовать dataset MovieLens
* Построить рекомендации (регрессия, предсказываем оценку) на фичах:
* TF-IDF на тегах и жанрах
* Средние оценки (+ median, variance, etc.) пользователя и фильма
* Оценить RMSE на тестовой выборке

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


Загрузим датасеты и посмотрим на них

In [2]:
movies = pd.read_csv('movies.csv')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings = pd.read_csv('ratings.csv')

In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
links = pd.read_csv('links.csv')

In [7]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [8]:
tags = pd.read_csv('tags.csv')

In [9]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


соединим вместе рейтинг, tags и кино

In [10]:
df = ratings.join(movies.set_index('movieId'),on='movieId')

In [11]:
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


Рассчитаем среднюю оценку, кол-во оценок и дисперссию

In [12]:
rate_movie = df.groupby('movieId').agg(['mean','count','var']).rating.reset_index()

In [13]:
rate_movie.sample(5)

,movieId,mean,count,var
6121,43912,2.000000,1,NaN
6456,52767,4.000000,3,0.000000
2016,2688,2.933333,15,0.995238
3822,5365,4.000000,1,NaN
7836,93790,4.000000,1,NaN


,movieId,mean,count,var
6121,43912,2.000000,1,NaN
6456,52767,4.000000,3,0.000000
2016,2688,2.933333,15,0.995238
3822,5365,4.000000,1,NaN
7836,93790,4.000000,1,NaN


Мы видим, что дисперсия по фильмам с одной оценкой равна NaN. Заменим на 0.

In [14]:
rate_movie['var'].fillna(0, inplace = True)

In [15]:
rate_movie.sample(5)

,movieId,mean,count,var
9689,187031,3.250000,2,1.125000
1768,2367,2.714286,14,0.873626
3910,5502,3.166667,63,0.967742
4855,7282,1.000000,1,0.000000
483,551,3.553763,93,0.986209


,movieId,mean,count,var
9689,187031,3.250000,2,1.125000
1768,2367,2.714286,14,0.873626
3910,5502,3.166667,63,0.967742
4855,7282,1.000000,1,0.000000
483,551,3.553763,93,0.986209


In [16]:
movie_genres = rate_movie.join(df.set_index('movieId'), on='movieId')

In [17]:
movie_genres.head()

,movieId,mean,count,var,userId,rating,timestamp,title,genres
0,1,3.92093,215,0.69699,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
0,1,3.92093,215,0.69699,5,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
0,1,3.92093,215,0.69699,7,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
0,1,3.92093,215,0.69699,15,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
0,1,3.92093,215,0.69699,17,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


,movieId,mean,count,var,userId,rating,timestamp,title,genres
0,1,3.92093,215,0.69699,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
0,1,3.92093,215,0.69699,5,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
0,1,3.92093,215,0.69699,7,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
0,1,3.92093,215,0.69699,15,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
0,1,3.92093,215,0.69699,17,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


Преобразуем поле genres

In [18]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [19]:
movie_genres['new_genres']  = movie_genres.apply(lambda row: change_string(row.genres),axis=1)

In [20]:
movie_genres.head()

,movieId,mean,count,var,userId,rating,timestamp,title,genres,new_genres
0,1,3.92093,215,0.69699,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Adventure Animation Children Comedy Fantasy
0,1,3.92093,215,0.69699,5,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Adventure Animation Children Comedy Fantasy
0,1,3.92093,215,0.69699,7,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Adventure Animation Children Comedy Fantasy
0,1,3.92093,215,0.69699,15,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Adventure Animation Children Comedy Fantasy
0,1,3.92093,215,0.69699,17,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Adventure Animation Children Comedy Fantasy


,movieId,mean,count,var,userId,rating,timestamp,title,genres,new_genres
0,1,3.92093,215,0.69699,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Adventure Animation Children Comedy Fantasy
0,1,3.92093,215,0.69699,5,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Adventure Animation Children Comedy Fantasy
0,1,3.92093,215,0.69699,7,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Adventure Animation Children Comedy Fantasy
0,1,3.92093,215,0.69699,15,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Adventure Animation Children Comedy Fantasy
0,1,3.92093,215,0.69699,17,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Adventure Animation Children Comedy Fantasy


дропнем старый столбец с жанрами

In [21]:
movie_genres.drop('genres',inplace=True,axis=1)

Соберем теги по каждом у фильму

In [22]:
all_tags = tags.groupby(by='movieId')[['tag']].agg(' '.join).reset_index()

In [23]:
data = all_tags.join(movie_genres.set_index('movieId'), on='movieId')

In [24]:
data.head()

,movieId,tag,mean,count,var,userId,rating,timestamp,title,new_genres
0,1,pixar pixar fun,3.92093,215.0,0.69699,1.0,4.0,9.649827e+08,Toy Story (1995),Adventure Animation Children Comedy Fantasy
0,1,pixar pixar fun,3.92093,215.0,0.69699,5.0,4.0,8.474350e+08,Toy Story (1995),Adventure Animation Children Comedy Fantasy
0,1,pixar pixar fun,3.92093,215.0,0.69699,7.0,4.5,1.106636e+09,Toy Story (1995),Adventure Animation Children Comedy Fantasy
0,1,pixar pixar fun,3.92093,215.0,0.69699,15.0,2.5,1.510578e+09,Toy Story (1995),Adventure Animation Children Comedy Fantasy
0,1,pixar pixar fun,3.92093,215.0,0.69699,17.0,4.5,1.305696e+09,Toy Story (1995),Adventure Animation Children Comedy Fantasy


,movieId,tag,mean,count,var,userId,rating,timestamp,title,new_genres
0,1,pixar pixar fun,3.92093,215.0,0.69699,1.0,4.0,9.649827e+08,Toy Story (1995),Adventure Animation Children Comedy Fantasy
0,1,pixar pixar fun,3.92093,215.0,0.69699,5.0,4.0,8.474350e+08,Toy Story (1995),Adventure Animation Children Comedy Fantasy
0,1,pixar pixar fun,3.92093,215.0,0.69699,7.0,4.5,1.106636e+09,Toy Story (1995),Adventure Animation Children Comedy Fantasy
0,1,pixar pixar fun,3.92093,215.0,0.69699,15.0,2.5,1.510578e+09,Toy Story (1995),Adventure Animation Children Comedy Fantasy
0,1,pixar pixar fun,3.92093,215.0,0.69699,17.0,4.5,1.305696e+09,Toy Story (1995),Adventure Animation Children Comedy Fantasy


присвоим тегам названия жанорв

In [25]:
data['tags'] = data.apply(lambda row: str(row.new_genres)+' '+ row.tag,axis=1)

In [26]:
data.drop(['tag','new_genres'],inplace=True,axis=1)

In [27]:
data.head()

,movieId,mean,count,var,userId,rating,timestamp,title,tags
0,1,3.92093,215.0,0.69699,1.0,4.0,9.649827e+08,Toy Story (1995),Adventure Animation Children Comedy Fantasy pi...
0,1,3.92093,215.0,0.69699,5.0,4.0,8.474350e+08,Toy Story (1995),Adventure Animation Children Comedy Fantasy pi...
0,1,3.92093,215.0,0.69699,7.0,4.5,1.106636e+09,Toy Story (1995),Adventure Animation Children Comedy Fantasy pi...
0,1,3.92093,215.0,0.69699,15.0,2.5,1.510578e+09,Toy Story (1995),Adventure Animation Children Comedy Fantasy pi...
0,1,3.92093,215.0,0.69699,17.0,4.5,1.305696e+09,Toy Story (1995),Adventure Animation Children Comedy Fantasy pi...


,movieId,mean,count,var,userId,rating,timestamp,title,tags
0,1,3.92093,215.0,0.69699,1.0,4.0,9.649827e+08,Toy Story (1995),Adventure Animation Children Comedy Fantasy pi...
0,1,3.92093,215.0,0.69699,5.0,4.0,8.474350e+08,Toy Story (1995),Adventure Animation Children Comedy Fantasy pi...
0,1,3.92093,215.0,0.69699,7.0,4.5,1.106636e+09,Toy Story (1995),Adventure Animation Children Comedy Fantasy pi...
0,1,3.92093,215.0,0.69699,15.0,2.5,1.510578e+09,Toy Story (1995),Adventure Animation Children Comedy Fantasy pi...
0,1,3.92093,215.0,0.69699,17.0,4.5,1.305696e+09,Toy Story (1995),Adventure Animation Children Comedy Fantasy pi...


Импортируем необходимые модули из sklearn

In [28]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
%matplotlib inline

In [29]:
tags = data.tags.tolist()
vec = CountVectorizer()
tf  = vec.fit_transform(tags)
tfidf = TfidfTransformer()
tfidf_ =tfidf.fit_transform(tf) 

In [30]:
df_tfidf=pd.DataFrame(tfidf_.toarray(),index=data.movieId).reset_index()

In [ ]:
df_full = data.merge(df_tfidf,on='movieId')

In [ ]:
df_full.head()

Сделаем предсказание для произвольного пользователя

In [ ]:
ratings.groupby('userId')['movieId'].count().sort_values(ascending=False)[:5]

Посмотрим топ фильмов, которые понравились пользователю 1912

In [ ]:
movies[movies.movieId.isin(ratings[ratings.userId==1912].sort_values(by='rating',ascending=False)[:50]['movieId'])]

Посмотрим на  пользователя 1912:

In [ ]:
for_predict = ratings[(ratings.userId==1912) & ratings.movieId.isin(data.movieId)].join(df_full.set_index('movieId'),on='movieId')

In [ ]:
for_predict.columns

In [ ]:
X = for_predict.drop(['userId','rating','timestamp','title','tags'],axis=1).set_index('movieId')
y = for_predict.rating

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
lm_lasso = Lasso()

In [ ]:
lm_lasso.fit(X_train,y_train)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test,lm_lasso.predict(X_test)))
rmse

Предскажем оценку фильмов, которые пользователь не оценивал, и покажем первые 10:

In [ ]:
reccomendation = df_full[(~df_full.movieId.isin(for_predict.movieId))].drop(['title','tags'],axis=1).set_index('movieId')

In [ ]:
reccomendation.dropna(inplace=True)

In [ ]:
reccomendation['rating'] = lm_lasso.predict(reccomendation)

In [ ]:
rec_list = reccomendation.sort_values('rating',ascending=False)[:10].index
rec_list

Выведем список наиболее актуальных фильмов:

In [ ]:
movies[movies.movieId.isin(rec_list)]